## Build Back Testing
Follow the buy and sell decisions from both the retail investors or the institution investors as per fund flow data provided, build two trading strategies Smart Retail Flow FT5009 and Smart Insti Flow FT5009. That is, you buy and sell the same stocks as retail investors or institutional investors from the fund flow report.

The strategy Smart Retail Flow FT5009 strictly follows the weekly buy sell decisions of retail investors and the strategy Smart Insti Flow FT5009 strictly follows the weekly buy sell decisions of institution investors.
- Prepare the return of the stocks (1%)
- Identify the position of stocks to buy and sell (2%)

close or open 

In [2]:
import pandas as pd
import numpy as np

top_stocks_path = './data/data.csv'
stock_prices_path = './data/stock.csv'
# 读取模拟的资金流数据
top_stocks_data = pd.read_csv(top_stocks_path)
stock_prices_data = pd.read_csv(stock_prices_path)
# stock_prices_data['Open Return'] = stock_prices_data.groupby('Stock Code')['Open'].pct_change()
PRICE = 'Open'

# 假设初始资金和相关参数
initial_capital = 1_000_000  # 初始资金1M
min_trade_units = 100        # 最小购买单位 100股
holding_period_limit = 4     # 最长持仓4周
borrow_cost_rate = 0.001     # 借贷利率 0.1%
weekly_capital_interval = 4  # 每四周重新计算交易资金
borrow_cash = 100_000        # 每次借贷现金 10w

# 创建两个策略组合
portfolio_retail = {}
portfolio_institution = {}

# 设置回测的日期和股票代码信息
dates = top_stocks_data['Date'].unique()

# 初始化组合
portfolio_retail[dates[0]] = {
    'cash': initial_capital,
    'short_sell_cash': 0, # 卖空的现金是锁定的，即每周的交易资金为cash的1/4，不带卖空的现金
    'position_units': {},
    'holding_weeks': {},
    'net_value': 0,
    'borrow_cash': 0,
}
portfolio_institution[dates[0]] = {
    'cash': initial_capital,
    'short_sell_cash': 0,
    'position_units': {},
    'holding_weeks': {},
    'net_value': 0,
    'borrow_cash': 0,
}

# 逐周执行买卖决策
for i, date in enumerate(dates):
    print(f"第{i+1}周：{date}")

    # 获取零售和机构的买卖信号
    retail_signals = top_stocks_data[(top_stocks_data['Date'] == date) & (top_stocks_data['Investor Type'] == 'retail')]
    insti_signals = top_stocks_data[(top_stocks_data['Date'] == date) & (top_stocks_data['Investor Type'] == 'institution')]

    # 处理每个策略的买卖信号
    for strategy, signals, portfolio in zip(
            ['Smart Retail Flow FT5009', 'Smart Insti Flow FT5009'],
            [retail_signals, insti_signals],
            [portfolio_retail, portfolio_institution]):
        
        # signals中的Stock Code在stock_prices_data中可能不存在，需要过滤掉
        signals = signals[signals['Stock Code'].isin(stock_prices_data['Stock Code'])]

        # 获取当前日期的组合
        if i > 0:
            # 复制上周的组合状态
            portfolio[date] = portfolio[dates[i-1]].copy()
        
        current_portfolio = portfolio[date]
        
        current_portfolio['net_value'] = 0  # 重置净值

        # 先花钱将卖空仓位的股票买回，并计算借贷成本（这一步可能造成short_sell_cash<0，因为强制一周时间就仓位归0，此时股价可能比short sell时更贵，低价卖出高价买入造成亏损，再加上借贷成本就更亏了）
        for stock_code, units in list(current_portfolio['position_units'].items()):
            if units < 0:  # 只有卖空仓位才计算借贷成本
                if stock_prices_data[(stock_prices_data['Date'] == dates[i - 1]) & (stock_prices_data['Stock Code'] == stock_code)].shape[0] > 0:  # 确保有上一周的数据
                    prev_date = dates[i - 1]
                    prev_price = stock_prices_data[(stock_prices_data['Date'] == prev_date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
                else:
                    prev_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
                cur_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
                avg_price = (prev_price + cur_price) / 2
                borrow_cost = abs(units) * avg_price * borrow_cost_rate
                current_portfolio['cash'] -= borrow_cost
                current_portfolio['short_sell_cash'] -= abs(units) * cur_price
                del current_portfolio['position_units'][stock_code]  # 清零卖空持仓

        # 如果short sell亏损，则将亏损的short sell用现金补上
        if current_portfolio['short_sell_cash'] < 0:
            current_portfolio['cash'] += current_portfolio['short_sell_cash']
            current_portfolio['short_sell_cash'] = 0


        if current_portfolio['cash'] < 0:
            # 如果现金不足，需要借贷(暂不考虑借贷利息)
            while current_portfolio['cash'] < 0:
                current_portfolio['cash'] += borrow_cash
                current_portfolio['borrow_cash'] += borrow_cash


        # 每周交易资金为总资金的1/4，每四周重新计算
        if i % weekly_capital_interval == 0:
            weekly_capital = current_portfolio['cash'] / weekly_capital_interval


        # 当现金不足时，卖出仓位最小的持仓，直到有足够的现金（>weekly_capital)，保证每周交易资金充足
        while current_portfolio['cash'] < weekly_capital:
            # 如果没有持仓，直接跳出
            if len(current_portfolio['position_units']) == 0:
                break
            # 卖出持仓最小的股票
            stock_code = min(current_portfolio['position_units'], key=current_portfolio['position_units'].get)
            units = current_portfolio['position_units'][stock_code]
            try:
                stock_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
            except:
                print(date, stock_code)
                ii = i
                tmp_date = dates[ii]
                while stock_prices_data[(stock_prices_data['Date'] == tmp_date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].shape[0] == 0:
                    ii -= 1
                    tmp_date = dates[ii]
                stock_price = stock_prices_data[(stock_prices_data['Date'] == tmp_date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]                    
            
            # stock_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
            sell_units = min(min_trade_units, abs(units))
            current_portfolio['cash'] += sell_units * stock_price
            current_portfolio['position_units'][stock_code] -= sell_units
            if current_portfolio['position_units'][stock_code] == 0:
                del current_portfolio['position_units'][stock_code]
                del current_portfolio['holding_weeks'][stock_code]
            else:
                # 不必更新持有时间，因为这只是为了弥补现金不足 
                pass


        # 分别计算买入和卖出的信号强度
        buy_signals = signals[signals['Action'] == 'buy']
        sell_signals = signals[signals['Action'] == 'sell']
        buy_signal_strength = buy_signals['Amount'].abs().sum()
        sell_signal_strength = sell_signals['Amount'].abs().sum()

        # 处理买入信号
        for _, signal in buy_signals.iterrows():
            stock_code = signal['Stock Code']
            amount = abs(signal['Amount'])
            signal_strength = amount / buy_signal_strength  # 买入信号强度

            trade_value = weekly_capital * signal_strength
            # 获取股票价格
            stock_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
            units = int(trade_value / stock_price / min_trade_units) * min_trade_units

            # 扩大持仓并记录持有时间
            if units > 0:
                total_cost = units * stock_price
                current_portfolio['cash'] -= total_cost
                current_portfolio['position_units'][stock_code] = current_portfolio['position_units'].get(stock_code, 0) + units
                current_portfolio['holding_weeks'][stock_code] = 1  # 重置持有时间
            # print('买了',units,'个: ', stock_code, current_portfolio)

        # 处理卖出信号
        for _, signal in sell_signals.iterrows():
            stock_code = signal['Stock Code']
            amount = abs(signal['Amount'])
            signal_strength = amount / sell_signal_strength  # 卖出信号强度

            trade_value = weekly_capital * signal_strength
            # 获取股票价格
            stock_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
            units = int(trade_value / stock_price / min_trade_units) * min_trade_units

            if units > 0:
                # 检查是否有持仓
                held_units = current_portfolio['position_units'].get(stock_code, 0)
                if held_units > 0:
                    # 先卖出持仓部分
                    sell_units = min(units, held_units)
                    current_portfolio['cash'] += sell_units * stock_price
                    current_portfolio['position_units'][stock_code] -= sell_units
                    if current_portfolio['position_units'][stock_code] == 0:
                        del current_portfolio['position_units'][stock_code]
                        del current_portfolio['holding_weeks'][stock_code]
                    else:
                        current_portfolio['holding_weeks'][stock_code] += 1
                   
                    # 剩余部分按short sell处理
                    if sell_units < units:
                        short_units = units - sell_units
                        current_portfolio['short_sell_cash'] += short_units * stock_price  # short sell收益计入现金
                        current_portfolio['position_units'][stock_code] = current_portfolio['position_units'].get(stock_code, 0) - short_units

                else:
                    # 没有持仓时执行short sell
                    current_portfolio['short_sell_cash'] += units * stock_price
                    current_portfolio['position_units'][stock_code] = current_portfolio['position_units'].get(stock_code, 0) - units
            # print('卖了',units,'个: ', stock_code, current_portfolio)

        # 更新持仓时间
        for stock_code in current_portfolio['position_units']:
            if stock_code not in buy_signals['Stock Code'].values and stock_code not in sell_signals['Stock Code'].values:
                current_portfolio['holding_weeks'][stock_code] += 1

        # 清理超过持仓期限的股票
        for stock_code, weeks in list(current_portfolio['holding_weeks'].items()):
            if weeks >= holding_period_limit:
                # 超过持仓期限，强制卖出剩余持仓
                units = current_portfolio['position_units'][stock_code]
                stock_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
                current_portfolio['cash'] += units * stock_price  # 卖出持仓

                del current_portfolio['position_units'][stock_code]
                del current_portfolio['holding_weeks'][stock_code]


        # 假设现在把所有的股票都换成现金，那么净值计算如下
        for stock_code, units in list(current_portfolio['position_units'].items()):
            stock_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
            if units < 0:
                # 确保有上一周的数据
                if stock_prices_data[(stock_prices_data['Date'] == dates[i - 1]) & (stock_prices_data['Stock Code'] == stock_code)].shape[0] > 0:
                    prev_date = dates[i - 1]
                    prev_price = stock_prices_data[(stock_prices_data['Date'] == prev_date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
                else:
                    prev_price = stock_prices_data[(stock_prices_data['Date'] == date) & (stock_prices_data['Stock Code'] == stock_code)][PRICE].values[0]
                avg_price = (prev_price + stock_price) / 2
                current_portfolio['net_value'] += units * avg_price * borrow_cost_rate

            current_portfolio['net_value'] += units * stock_price

        current_portfolio['net_value'] += (current_portfolio['cash'] + current_portfolio['short_sell_cash'] - current_portfolio['borrow_cash'])
        
        print(f"{strategy} Portfolio cash: {current_portfolio['cash']:.2f}, short_sell_cash: {current_portfolio['short_sell_cash']:.2f}, borrow_cash: {current_portfolio['borrow_cash']:.2f}, net_value: {current_portfolio['net_value']:.2f}")
        # 格式化输出持仓为正和负的股票
        positive_positions = {k: v for k, v in current_portfolio['position_units'].items() if v > 0}
        negative_positions = {k: v for k, v in current_portfolio['position_units'].items() if v < 0}

        positive_df = pd.DataFrame(list(positive_positions.items()), columns=['Stock Code', 'Positive Units'])
        negative_df = pd.DataFrame(list(negative_positions.items()), columns=['Stock Code', 'Negative Units'])

        # 合并两个 DataFrame
        merged_df = pd.merge(positive_df, negative_df, on='Stock Code', how='outer')

        # 填充 NaN 值为 0
        merged_df = merged_df.fillna(np.nan)

        # 格式化输出
        print("Combined Positions:")
        print(merged_df.to_string(index=False))
        print()
       
print("Retail Portfolio Final:", portfolio_retail[dates[-1]])
print("Institution Portfolio Final:", portfolio_institution[dates[-1]])

第1周：2019-01-07
Smart Retail Flow FT5009 Portfolio cash: 750604.09, short_sell_cash: 245325.12, borrow_cash: 0.00, net_value: 998362.18
Combined Positions:
Stock Code  Positive Units  Negative Units
       544         27600.0             NaN
       5CP         56800.0             NaN
       BDX        449900.0             NaN
       BN4             NaN         -5100.0
       C09             NaN         -2400.0
      C38U         12600.0             NaN
       CC3         11300.0             NaN
      CJLU         22900.0             NaN
       D05             NaN         -3400.0
       G13             NaN        -15400.0
       H02          1200.0             NaN
       O39             NaN         -6900.0
       U11             NaN         -1900.0
       V03             NaN         -1400.0
       Y92             NaN      -4825600.0
       Z74             NaN         -7800.0

Smart Insti Flow FT5009 Portfolio cash: 752916.25, short_sell_cash: 249125.05, borrow_cash: 0.00, net_value: 9997

/var/folders/x2/f0lc5ft50x9d3rhmc18zp8240000gn/T/ipykernel_76023/4165186804.py:243: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(np.nan)


Smart Insti Flow FT5009 Portfolio cash: 1028775.88, short_sell_cash: 546778.40, borrow_cash: 0.00, net_value: 2839772.50
Combined Positions:
Stock Code  Positive Units  Negative Units
      A17U         75000.0             NaN
      A7RU         27100.0             NaN
       AGS          9300.0             NaN
      AJBU          9100.0             NaN
       AWX          6100.0             NaN
       BN4         10800.0             NaN
       BS6          5300.0             NaN
       BSL         15000.0             NaN
      BUOU         42200.0             NaN
       C09          5300.0             NaN
      C38U         16300.0             NaN
       C52             NaN         -6400.0
       C6L         33600.0             NaN
      CJLU             NaN        -36700.0
       D05          7200.0             NaN
       F34          6000.0             NaN
       G13         95300.0             NaN
       HMN             NaN         -9800.0
       J36          2500.0             NaN

/var/folders/x2/f0lc5ft50x9d3rhmc18zp8240000gn/T/ipykernel_76023/4165186804.py:243: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(np.nan)


Smart Insti Flow FT5009 Portfolio cash: 1587526.76, short_sell_cash: 586117.82, borrow_cash: 0.00, net_value: 3685022.15
Combined Positions:
Stock Code  Positive Units  Negative Units
       1MZ             NaN        -51500.0
       9CI             NaN         -9400.0
      A17U         13400.0             NaN
       AP4         13100.0             NaN
       AWX             NaN         -6600.0
       BN4          7300.0             NaN
       BS6         99100.0             NaN
       BSL             NaN        -11900.0
      BUOU         10800.0             NaN
       C07           900.0             NaN
       C52         23400.0             NaN
       C6L             NaN         -5100.0
       D05         12500.0             NaN
       H78             NaN         -4500.0
      J69U             NaN         -3400.0
      M44U         17600.0             NaN
      N2IU         33600.0             NaN
       O39         36600.0             NaN
       S58          4000.0             NaN

/var/folders/x2/f0lc5ft50x9d3rhmc18zp8240000gn/T/ipykernel_76023/4165186804.py:243: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(np.nan)


Smart Insti Flow FT5009 Portfolio cash: 679607.23, short_sell_cash: 658512.64, borrow_cash: 0.00, net_value: 4418971.37
Combined Positions:
Stock Code  Positive Units  Negative Units
       5E2         58600.0             NaN
       9CI          5800.0             NaN
      A17U             NaN        -19800.0
       BN4          3100.0             NaN
      BUOU             NaN        -20400.0
      C38U             NaN         -9400.0
       C52          1900.0             NaN
       C6L          2200.0             NaN
       D05         12700.0             NaN
       F34             NaN         -3700.0
       G13         24900.0             NaN
       H78             NaN         -1900.0
      J69U             NaN        -13400.0
      JYEU             NaN        -13000.0
      ME8U             NaN         -5800.0
      N2IU         19900.0             NaN
       O39          9200.0             NaN
       S58         60700.0             NaN
       S63          1200.0             NaN
